# Developing Optimisers in Python
Jay Parfitt 2013962

## Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Task 1
Implement the functions

Global counter for the function evaluations

In [3]:
funcCounters = {'f':0, 'g1':0, 'g2':0, 'g3':0, 'g4':0}

Defining the functions

In [4]:
#f(x)
def f(x):
  global funcCounters
  funcCounters['f'] += 1
  return x[0]**2 * x[1] * (2 + x[2])

#g1(x)
def g1(x):
  global funcCounters
  funcCounters['g1'] += 1
  return 1 - (x[1]**2 * x[2]) / (71785 * x[0]**4)

#g2(x)
def g2(x):
  global funcCounters
  funcCounters['g2'] += 1
  part1 = 4 * x[1]**2 - x[0] * x[1]
  part2 = 12566 * (x[1] * x[0]**3 - x[0]**4)
  part3 = 1 / (5108 * x[0]**2)
  return part1 / part2 + part3 - 1

#g3(x)
def g3(x):
  global funcCounters
  funcCounters['g3'] += 1
  return 1 - (140.45 * x[0]) / (x[1]**2 * x[2])

#g4(x)
def g4(x):
  global funcCounters
  funcCounters['g4'] += 1
  return ((x[0] + x[1]) / 1.5) - 1

In [7]:
x = np.array([0.42, 0.9, 7])
print("Objective function output: ", f(x))
print("First contraint function output: ", g1(x))
print("Second contraint function output: ", g2(x))
print("Third contraint function output: ", g3(x))
print("Fourth contraint function output: ", g4(x))

Objective function output:  1.4288399999999999
First contraint function output:  0.9974616459784045
Second contraint function output:  -0.9924857111736846
Third contraint function output:  -9.403703703703702
Fourth contraint function output:  -0.12


## Task 2
Implement Random Search

### Feasability function
Checks if a solution x satisfies all contraintes and bounds

In [8]:
def isFeasable(x):
  return all([
      g1(x) <= 0,
      g2(x) <= 0,
      g3(x) <= 0,
      g4(x) <= 0,
      0.05 <= x[0] <= 2,
      0.25 <= x[1] <= 1.3,
      2 <= x[2] <= 15
  ])

### Random Search function


In [22]:
def randomSearch(maxEvals=500, randomSeed=42):
  # Setting random seed
  np.random.seed(randomSeed)
  # initialising best solution
  bestX, bestVal = None, float('inf')

  # setting the bounds
  for _ in range(maxEvals):
    x = np.array([
        np.random.uniform(0.05, 2),
        np.random.uniform(0.25, 1.3),
        np.random.uniform(2, 16)
    ])

    # rejects and regenerates if constraints are not satisfied
    if isFeasable(x):
      val = f(x)
      if val < bestVal or bestVal is None:
        bestVal = val
        bestX = x

  return bestX, bestVal

In [23]:
bestX, bestVal = randomSearch()
print("Best solution: ", bestX)
print("Best value: ", bestVal)

Best solution:  [ 0.07144187  0.39372991 14.60026099]
Best value:  0.03335945160892047


## Task 3
Implement the simulated annealing method

### Generate Initial Solution

Inital solution is also required to be feasable to allow the algorithm to start from a valid point

In [26]:
def generateInitialSolution():
  while True:
    # generates random solution within bounds
    x = np.array([
        np.random.uniform(0.05, 2),
        np.random.uniform(0.25, 1.3),
        np.random.uniform(2, 16)
    ])

    # checks if it's feasable
    if isFeasable(x):
      return x

### Simulated Annealing function

In [29]:
def simulatedAnnealing(maxEvals=500, initalTemp=100, coolingRate=0.95, randomSeed=42):
  np.random.seed(randomSeed)
  x = generateInitialSolution()

  bestX, bestVal = x, f(x)
  currentX, currentVal = x, f(x)
  temp = initalTemp

  for _ in range(maxEvals):
    neighbour = currentX + np.random.uniform(-0.1, 0.1, size=3)
    neighbour[0] = np.clip(neighbour[0], 0.05, 2)
    neighbour[1] = np.clip(neighbour[1], 0.25, 1.3)
    neighbour[2] = np.clip(neighbour[2], 2, 15)

    if isFeasable(neighbour):
      neighbourVal = f(neighbour)
      delta = neighbourVal - currentVal

      # accepts neighbour based on SA criteria
      if delta < 0 or np.exp(-delta / temp) > np.random.rand():
        currentX, currentVal = neighbour, neighbourVal
        if neighbourVal < bestVal or bestVal is None:
          bestX, bestVal = neighbour, neighbourVal

    temp *= coolingRate

  return bestX, bestVal


In [30]:
initialSolution = generateInitialSolution()
print("Initial solution: ", initialSolution)
print("Initial value: ", f(initialSolution))

bestX, bestVal = simulatedAnnealing()
print("Best solution: ", bestX)
print("Best value: ", bestVal)

Initial solution:  [0.07282789 0.69313597 6.13056991]
Initial value:  0.029890617933438286
Best solution:  [0.07380479 0.87339854 4.25139232]
Best value:  0.029741184767636847


## Task 4
Performance Evaluation